In [1]:
pip install transformers[torch] tokenizers datasets evaluate rouge_score sentencepiece huggingface_hub --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 4

In [2]:
pip install datasets

In [3]:
# Import the necessary libraries
import nltk
from datasets import load_dataset
import evaluate
import pandas as pd
import numpy as np
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [4]:
dataset = load_dataset("gjyotk/demo")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction (string)', 'output (string)'],
        num_rows: 530
    })
    test: Dataset({
        features: ['instruction (string)', 'output (string)'],
        num_rows: 45
    })
})

In [6]:
train= dataset['train']
test= dataset['test']

In [7]:
train

Dataset({
    features: ['instruction (string)', 'output (string)'],
    num_rows: 530
})

In [8]:
test

Dataset({
    features: ['instruction (string)', 'output (string)'],
    num_rows: 45
})

In [9]:
type(train)

datasets.arrow_dataset.Dataset

In [10]:
# Load the tokenizer, model, and data collator
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [11]:
train

Dataset({
    features: ['instruction (string)', 'output (string)'],
    num_rows: 530
})

In [12]:
# We prefix our tasks with "answer the question"
prefix = "answer the question: "

# Define our preprocessing function
def preprocess_function(examples):
    """Add prefix to the sentences, tokenize the text, and set the labels"""
    # The "inputs" are the tokenized answer:
    inputs = [prefix + doc for doc in examples["instruction (string)"]]
    model_inputs = tokenizer(inputs, max_length= 128, truncation=True)

    # The "labels" are the tokenized outputs:
    labels = tokenizer(text_target=examples["output (string)"], max_length= 320, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Map the preprocessing function across our dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/530 [00:00<?, ? examples/s]

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

##What is ROUGE score?
ROUGE stands for Recall-Oriented Understudy for Gisting Evaluation. Some key components of ROUGE for question-answering include:

ROUGE-L: Measures the longest common subsequence between the candidate and reference answers. This focuses on recall of the full text.
ROUGE-1, ROUGE-2, ROUGE-SU4: Compare unigram, bigram, 4-gram overlaps between candidate and reference. Focus on recall of key parts/chunks
Higher ROUGE scores generally indicate better performance for question answering. Scores close to or above 0.70+ are considered strong
When using this metric, processing like stemming, and removing stopwords can help improve the overall performance
With this understanding, the following helper function compute_metrics can help compute the underlying ROUGE score. Prior to the implementation of the function, it is necessary to set up ROUGE and NLTK.

In [14]:
# We will use the Rouge score to evaluate the training progress. The Rouge score basically compares the n-grams, or word combinations, in the generated text with the n-grams in the target text.

# Set up Rouge score for evaluation
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # rougeLSum expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return result

**Training Loss and Validation Loss:** Lower values in these metrics are preferable as they indicate a better fit of the model to the data. The training and validation loss has decreased over the epochs, with the lowest values recorded in epoch 3.

**Rouge Metrics (Rouge1, Rouge2, Rougel, and Rougelsum)**: Higher values in these metrics are preferable as they indicate better text summarization performance. Across all four Rouge metrics, the values have increased over the epochs, with the highest values recorded in epoch 3.

In [15]:
# Set up training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=0,
    num_train_epochs=5,
    predict_with_generate=True,
    push_to_hub=False
)

# Set up trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,1.466681,0.368152,0.174252,0.329481,0.329829
2,No log,1.236052,0.403503,0.215851,0.366553,0.364698
3,No log,1.136852,0.448226,0.276460,0.397091,0.396555
4,No log,1.099614,0.452561,0.292618,0.410640,0.412568
5,No log,1.108637,0.448693,0.280873,0.409729,0.410961


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=335, training_loss=1.4534026644123135, metrics={'train_runtime': 137.495, 'train_samples_per_second': 19.273, 'train_steps_per_second': 2.436, 'total_flos': 94247755130880.0, 'train_loss': 1.4534026644123135, 'epoch': 5.0})

In [16]:
from huggingface_hub import login, notebook_login

In [17]:
# write token used
notebook_login('input write token here')

In [19]:
# Save our tokenizer and create model card
tokenizer.save_pretrained('gjyotk/flan-t5-base-whrs-2')

#Create model card
trainer.create_model_card()

# Push the results to the hub
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

events.out.tfevents.1712067650.c3d0f9b1aad3.533.0:   0%|          | 0.00/8.33k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/gjyotk/results/commit/42a9de4ef74a428f932c9951952af405b607c653', commit_message='End of training', commit_description='', oid='42a9de4ef74a428f932c9951952af405b607c653', pr_url=None, pr_revision=None, pr_num=None)

In [20]:
#Read token
notebook_login('input read token here')

In [21]:
from transformers import pipeline

In [25]:

# load model and tokenizer from huggingface hub with pipeline
qna = pipeline("text2text-generation", model="gjyotk/results")

config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [26]:
# testing
ques='what is PCOS?'
res = qna(ques)

print("ques:", ques, "\nans: ", res)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


ques: what is PCOS? 
ans:  [{'generated_text': 'PCOS is a hormonal disorder characterized by heavy menstrual bleeding, acne, and'}]


In [27]:
# testing
ques='what is the role of schools in menstrual awareness?'
res = qna(ques)

print("ques:", ques, "\nans: ", res)

ques: what is the role of schools in menstrual awareness? 
ans:  [{'generated_text': 'They can provide educational materials, provide menstrual health education, and promote gender equality.'}]


In [30]:
# testing
ques='How long menstruation last in women?'
res = qna(ques)

print("ques:", ques, "\nans: ", res)

ques: How long menstruation last in women? 
ans:  [{'generated_text': 'Menstruation typically lasts for about 3 to 7 days in women.'}]


In [31]:
# testing
ques='How long is a menstrual cycle in women?'
res = qna(ques)

print("ques:", ques, "\nans: ", res)

ques: How long is a menstrual cycle in women? 
ans:  [{'generated_text': 'A menstrual cycle in women typically lasts for about 14 days, although it can range'}]


In [33]:
# testing
ques='what is ovulation?'
res = qna(ques)

print("ques:", ques, "\nans: ", res)

ques: what is ovulation? 
ans:  [{'generated_text': 'Ovulation is the process by which a mature egg is released from the ovary'}]
